In [5]:
import math
import json

import utils.components as comps
import utils.instrument as instr
import utils.functions as funcs

t_cal = comps.TraceDataCalculator()

sa = None

with open(r'config\giteki_tmp.json', 'r') as f:
    giteki_dict = json.load(f)

In [6]:
def unit_measurement(sa: instr.SpectrumAnalyzer, sa_config: dict):
    # display sa config
    funcs.show_configure(sa_config)

    sa.config(param='continues_sweep', value='OFF')  # stop sweep
    input('Waiting for board launch up. Press enter to start measurement.')

    sa.config(param=sa_config)  # start continues sweep

    input(f"Waiting trace to stabilize. Press Enter to stop sweep.")
    sa.config(param='continues_sweep', value='OFF')  # stop sweep

    peak_freq, peak_power = sa.aquire_peak_point()

    return peak_freq, peak_power

In [11]:
def measure_obw_and_sbw(sa, giteki_dict: dict, rule='49_27_3'):
    # calculate center freq. and span
    start_freq = float(giteki_dict['masks'][rule]['obw'].split('~')[0])  #GHz
    stop_freq = float(giteki_dict['masks'][rule]['obw'].split('~')[1])   #GHz
    sbw_limit = float(giteki_dict['masks']['sbw']) * 1e3  # MHz
    obw_limit = (stop_freq - start_freq) * 1e3  # MHz
    center_freq = (start_freq + stop_freq) / 2  # GHz
    
    # place tmp maker for obw measurement
    sa.config(param='obw_percent', value='99PCT')
    sa.config(param='obw_measure', value='SEL OBW', delimiter=':')
    sa.config(param='obw_measure', value='RES? OBW', delimiter=':')

    # construct other cfg of spectrum analyzer
    sa_cfg = {'center_freq': f"{center_freq}GHz", 'span': f"{obw_limit * 3}MHz"}
    sa_cfg.update(giteki_dict['obw_and_sbw'])

    unit_measurement(sa, sa_cfg)          # perform measurement
    sa.save_screenshot('obw_and_sbw.png')    # save screenshot
    sa.save_trace_to_csv('obw_and_sbw.csv')  # save trace data

    # use trace data to calculate obw and sbw
    global t_cal
    t_cal.read_trace_data('obw_and_sbw.csv')

    # calculate obw
    obw, l_freq, r_freq = t_cal.calculate_obw()
    obw_status = "Passed" if obw <= obw_limit else "Failed"
    l_freq_status = "Passed" if l_freq >= start_freq else "Failed"
    r_freq_status = "Passed" if r_freq <= stop_freq else "Failed"
    # calculate abw
    sbw = t_cal.calculate_sbw()
    sbw_status = "Passed" if sbw >= sbw_limit else "Failed"

    return {
        'obw': [f"{obw}MHz", obw_status],
        'obw lower freq': [f"{l_freq}GHz", l_freq_status],
        'obw upper freq': [f"{r_freq}GHz", r_freq_status],
        'sbw': [f"{sbw}MHz", sbw_status]
    }


SyntaxError: unterminated string literal (detected at line 33) (3985966272.py, line 33)

In [13]:
def measure_peak_power(sa: instr.SpectrumAnalyzer, giteki_dict: dict, rule='49_27_3', method='general'):
    freq_interval = giteki_dict['masks'][rule]['obw']
    start_freq = f"{freq_interval.split('~')[0]}GHz"
    stop_freq = f"{freq_interval.split('~')[1]}GHz"

    # construct spectrum analyzer config
    sa_cfg = {'start_freq': start_freq, 'stop_freq': stop_freq}
    sa_cfg.update(giteki_dict['search']['common'])
    sa_cfg.update(giteki_dict['search']['diff']['peak']['general'])  # set RBW to 3MHz, VBW to 10MHz

    search_freq, _ = unit_measurement(sa, sa_cfg)
    sa.save_screenshot(f'search_peak.png')

    # zoom in with span 100MHz
    del sa_cfg['start_freq']
    del sa_cfg['stop_freq']
    sa_cfg.update({'center_freq': search_freq, 'span': '100MHz'})
    if method != 'general':
        # wide RBW to 50MHz
        sa_cfg.update(giteki_dict['search']['diff']['peak']['exception'])
    
    search_freq, search_peak = unit_measurement(sa, sa_cfg)
    sa.save_screenshot(f'search_span=100MHz_peak.png')

    # calculate correction factor
    rbw = float(sa_cfg['RBW'].split('MHz'))
    corr_factor = 20 * math.log10(float(50.0) / rbw)
    peak = corr_factor + search_peak

    # load limit
    obw_key = giteki_dict['masks'][rule]['obw']
    limit = giteki_dict['masks'][rule][obw_key]['peak']
    dev = giteki_dict['masks'][rule][obw_key]['allowable_dev']
    limit = 10 * math.log10((10 ** (limit / 10)) * (1 + dev))

    status = 'Passed' if peak <= limit else 'Failed'

    return {
        'reading value': f"{round(search_peak, 2)}dBm",
        'freq.': f"{round(search_freq / 1e9, 5)}GHz",
        'corr. factor': f"{round(corr_factor, 2)}",
        'peak (reading val. + corr. factor)': [f"{round(peak, 2)}dBm", status]
    }

In [6]:
def measure_ave_power(sa: instr.SpectrumAnalyzer, giteki_dict: dict, rule='49_27_3', method='general'):
    freq_interval = giteki_dict['masks'][rule]['obw']
    start_freq = f"{freq_interval.split('~')[0]}GHz"
    stop_freq = f"{freq_interval.split('~')[1]}GHz"

    # construct spectrum analyzer config
    sa_cfg = {'start_freq': start_freq, 'stop_freq': stop_freq}
    sa_cfg.update(giteki_dict['search']['common'])

    search_freq, _ = unit_measurement(sa, sa_cfg)
    sa.save_screenshot(f'search_ave.png')
    sa.save_trace_to_csv(f'trace_{freq_interval}.csv')  # to draw plot

    # zoom in with span 100MHz and 10MHz
    del sa_cfg['start_freq']
    del sa_cfg['stop_freq']
    for span in ['100MHz', '10MHz']:
        sa_cfg.update({'center_freq': search_freq, 'span': span})
        search_freq, _ = unit_measurement(sa, sa_cfg)
        sa.save_screenshot(f'search_span={span}_ave.png')

    # measure step
    sa_cfg.update({'center_freq': search_freq})
    if method == 'general':
        sa_cfg.update(giteki_dict['measure']['common'])
        measure_freq, _ = unit_measurement(sa, sa_cfg)
        sa.save_trace_to_csv('measure_peak.csv')

        # use trace data to calculate average power
        global t_cal
        t_cal.read_trace_data('measure_peak.csv')
        ave = t_cal.calculate_ave_power()

    else:
        sa_cfg.update(giteki_dict['measure']['diff']['ave'])
        measure_freq, ave = unit_measurement(sa, sa_cfg)

    sa.save_screenshot(f'measure_ave_{method}.png')
    
    # load limit
    obw_key = giteki_dict['masks'][rule]['obw']
    d_freq = giteki_dict['masks'][rule][obw_key]['dividing_freq']
    idx = '1' if measure_freq <= d_freq else '2'
    limit = giteki_dict['masks'][rule][obw_key][f'ave{idx}']
    dev = giteki_dict['masks'][rule][obw_key]['allowable_dev']
    limit = 10 * math.log10((10 ** (limit / 10)) * (1 + dev))

    status = 'Passed' if ave <= limit else 'Failed'

    return {
        'ave power': f"{round(ave, 2)}dBm",
        'freq': f"{round(measure_freq / 1e9, 5)}GHz"
    }


In [ ]:
def measure_spurious(sa: instr.SpectrumAnalyzer, giteki_dict: dict, rule='49_27_3'):
    result = {}
    freq_intervals = giteki_dict['masks'][rule].copy()
    
    del freq_intervals[giteki_dict['masks'][rule]['obw']]
    del freq_intervals['obw']

    # iterate all spurious freq. band
    for freq_interval in freq_intervals:
        start_freq = f"{freq_interval.split('~')[0]}GHz"
        stop_freq = f"{freq_interval.split('~')[1]}GHz"

        # search step
        sa_cfg = {'start_freq': start_freq, 'stop_freq': stop_freq}
        sa_cfg.update(giteki_dict['search']['common'])

        search_freq, search_peak = unit_measurement(sa, sa_cfg)
        sa.save_screenshot(f'search_spurious_{freq_interval}.png')
        sa.save_trace_to_csv(f'trace_{freq_interval}.csv')  # to draw plot

        ave_limit = giteki_dict['masks'][rule][freq_interval]['ave']
        peak_limit = giteki_dict['masks'][rule][freq_interval]['peak']

        peak = ave = cal_ave = peak_freq = ave_freq = None
        if search_peak <= ave_limit:
            ave, ave_freq = search_peak, search_freq

        if search_peak <= peak_limit - 3:
            peak, peak_freq = search_peak, search_freq

        if peak is None or ave is None:  
            del sa_cfg['start_freq']
            del sa_cfg['stop_freq']

            for span in ['100MHz', '10MHz']:  # zoom in step
                sa_cfg.update({'center_freq': search_freq, 'span': span})
                search_freq, search_peak = unit_measurement(sa, sa_cfg)
                sa.save_screenshot(f'spurious_span={span}_{freq_interval}.png')

            if peak is None:  # update peak result if necessary
                peak, peak_freq = search_peak, search_freq
            
            # measure step (if necessary)
            if ave is None:
                sa_cfg.update({'center_freq': search_freq})

                sa_cfg.update(giteki_dict['measure']['common'])
                ave_freq, ave = unit_measurement(sa, sa_cfg)

                # save trace data to get calculate average
                sa.save_trace_to_csv(f'spurious_ave_measure_{freq_interval}.csv')
                global t_cal
                t_cal.read_trace_data(f'spurious_ave_measure_{freq_interval}.csv')
                cal_ave = t_cal.calculate_ave_spurious()

        # pass / fail determination
        peak_status = 'Passed' if peak <= peak_limit else 'Failed'
        m_ave_status = 'Passed' if ave <= ave_limit else 'Failed'
        if cal_ave:
            c_ave_status = 'Passed' if cal_ave <= ave_limit else 'Failed'
        else:
            c_ave_status = 'None'

        result[freq_interval] = {
            'peak': [f'{round(peak)}dBm', peak_status],
            'peak freq': f'{round(peak_freq) / 1e9}GHz',
            'ave (reading val)': [f'{round(ave)}dBm', m_ave_status],
            'ave (cal.)': [f'{round(cal_ave)}dBm', c_ave_status],
            'ave freq.': f'{round(ave_freq) / 1e9}GHz'
        }

    return result
